In [1]:
import math
import re
import heapq
import csv

import pandas as pd
import numpy as np

import time

In [2]:
import numpy.linalg as LA

from IPython.display import clear_output
from fingerprint.fingerprint import get_fingerprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


## Settings

In [3]:
input_filename = 'data/unique_sorted_split.txt' 

#output_filename = 'output/clustered.txt'
mapping_filename = 'output/mapping_clusters.csv'
label_filename = 'output/labels_clusters.csv'

threshold = 0.85


## Load data

In [4]:
df = pd.read_csv(input_filename, names=["Organization"], quoting=csv.QUOTE_NONE)


## Run Tf Idf Clustering

In [5]:
id_table = {}
ID = 0

def get_id(label, label_file):
    global ID
    if label not in id_table:
        ID += 1        
        id_table[label] = ID
        label_file.write(f"{ID},{label}")
    return id_table[label]
    

In [ ]:
labels = open(label_filename, "w")
mapping = open(mapping_filename, "w")

vectorizer = TfidfVectorizer()
corpus = set()

start = time.time()

for i, s in df.iterrows():
    if i % 10000 == 0:
        clear_output()
        print(f"{i} / {df.shape[0]}\nElapsed: {time.time()-start}s")
        
    original_string = s.Organization
    cleaned_string = get_fingerprint(original_string)
    
    if len(cleaned_string.split()) < 2:
        mapping.write(f"{original_string},-1\n")
        continue
    
    if not corpus: # Create initial cluster
        label = cleaned_string
        corpus = set()
        corpus.update(cleaned_string.split())
    else:
        train = vectorizer.fit_transform([' '.join(corpus)]).toarray()
        test = vectorizer.transform([cleaned_string]).toarray() # Should this be original or cleand label?
        score = (np.inner(train, test)/(LA.norm(train)*LA.norm(test)))[0][0]

        if np.isnan(score) or score < threshold: # Create new cluster
            corpus = set()
            corpus.update(cleaned_string.split())
            train = vectorizer.fit_transform([' '.join(corpus)]).toarray()
            test = vectorizer.transform([cleaned_string]).toarray() # Should this be original or cleand label?
            score = (np.inner(train, test)/(LA.norm(train)*LA.norm(test)))[0][0]
            label = cleaned_string

    #labels.write(f"{get_id(label, l)},{label}\n")
    mapping.write(f"{original_string},{get_id(label, labels)}\n")

labels.close()
mapping.close()

310000 / 3261955
Elapsed: 1372.5081799030304s
